# Football Project 2

This porject will take different data set to compute the probability of the football prediction. Previously we used past matches data and not player data.

This point we would like the improve the prediction of accuracy from (40% - 50%) to (60% - 70%) accuracy.

The data input plan:
- getting player stats from the Fifa 21 ( web scraping )
- using news roster to combined the player stats ( web scraping )



# Inputting Player Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\nghen\OneDrive\Desktop\Football project 2\players_21.csv")
pd.options.display.max_columns = None


# Cleaning player dataframe 

In [3]:
del df['player_url']
del df['age']
del df['dob']
del df['height_cm']
del df['weight_kg']
del df['league_rank']
del df['value_eur']
del df['wage_eur']
del df['player_positions']
del df['international_reputation']
del df['weak_foot']
del df['skill_moves']
del df['work_rate']
del df['body_type']
del df['real_face']
del df['release_clause_eur']
del df['player_tags']
del df['loaned_from']
del df['joined']
del df['contract_valid_until']
del df['nation_position']
del df['nation_jersey_number']
del df['player_traits']

In [4]:
df = df.fillna(0)

## Football Odd data

In [5]:
data_odd = pd.read_csv(r"C:\Users\nghen\OneDrive\Desktop\E0.csv")

In [6]:
football_team = []
for i in data_odd['HomeTeam']:
    if i in football_team:
        continue
    else:
        football_team.append(i)


In [7]:
football_team_code = [133600, 133632, 133602, 133636, 133611, 133616, 133619, 133811,133615, 133635, 133612, 133604, 134778, 134777, 133610, 133626, 133601, 133599, 133623, 133613 ]

In [8]:
football_team_tuple = tuple(zip(football_team, football_team_code))

In [9]:
home_team_code_list = []
for i in data_odd['HomeTeam']:

    for x in football_team_tuple:
        if i in x:
            home_team_code_list.append(x[1])
            
away_team_code_list = []
for i in data_odd['AwayTeam']:
    for x in football_team_tuple:
        if i in x:
            away_team_code_list.append(x[1])         

In [10]:
data_odd['HomeCode'] = home_team_code_list
data_odd['AwayCode'] = away_team_code_list
data_odd = data_odd.drop(columns = data_odd.iloc[:,0:24])


## Scraping website data (2020/2021)


In [11]:
from bs4 import BeautifulSoup as bs
import time 
import requests
import random 

In [12]:
from urllib.request import urlopen, Request

url = 'https://www.thesportsdb.com/season.php?l=4328&s=2020-2021'
req = requests.get(url)
soup =bs(req.text, 'html.parser')

In [13]:
link_list = []
for link in soup.find_all('a'):
    link_list.append(link.get('href'))


In [14]:
match_id = []

for i in link_list:
    if i.startswith('../event/'):
        x = i[9:]
        if x not in match_id:
            match_id.append(i[9:])
        
       

In [15]:
def homeg(home, roles):
    team_info_home = []
    for i in home:
        x_finding = i.findAll(text=True)
        for list_ in x_finding:
            if list_ == roles:
                list_.clear()
            else:
                name = ''.join(list_)
                team_info_home.append(name)

    home_team = []
    for i in team_info_home:
        x = i.strip()
        home_team.append(x)

    home_team = list(filter(None,home_team))

    empty =[]
    special_bin = []
    home_team_player =[]
    for i in home_team:
        if i not in roles:
            empty.append(i)
            if i.isdigit() == True:
                empty.clear()
        else:
            home_team_player.append(' '.join(empty))
            empty.clear()
    return home_team_player

In [16]:
# team name 
def code(soup_player):
    binn = soup_player.find(class_='col-sm-3')
    bin_x = binn.findAll('a')
    bin_y = []
    for i in bin_x:
        make_string = str(i)
        bin_y.append(make_string)

    keyword = 'team'
    team_code = []

    for i in bin_y:
        if keyword in i:
            team_code.append(i)

    Teams_code = []
    empty_code = []

    for i in team_code:
        empty_code.clear()
        for char in i:
            if char.isdigit():
                empty_code.append(char)
        Teams_code.append(''.join(empty_code))       

    return Teams_code

In [17]:
# the score board
def scores(soup_score):
    soup_score.findAll("td")
    score_list = []
    for i in soup_score.find_all('h2'):
        score_list.append(i)

    str_list = []
    for i in score_list:
        str_list.append(str(i))
    
    del str_list[0]   

    score = []
    for i in str_list:
        x = int(float(i[4]))
        score.append(x)
    return score

In [18]:
url_root = 'https://www.thesportsdb.com/event/'

full_home_team = []
full_away_team = []
score_board = []
for i in match_id:
    url_player = url_root + i
    req_player = requests.get(url_player)
    soup_player = bs(req_player.text, 'html.parser')
    rrrub = soup_player.find(class_='col-sm-9')
    rub = rrrub.findAll('table')
    
    try:
        home = rub[3].findAll('td')
        away = rub[4].findAll('td')
        roles = ['Defender', 'Midfielder', 'Defensive Midfielder', 'Left Back', 'Right Back', 'Goalkeeper',
                 'Forward', 'Right Wing', 'Left Wing', 'Centre Back', 'Centre Midfielder', 'Attacking Midfielder', 'Winger']
        
        home_team_player = homeg(home, roles) # generate list of home players name
        away_team_player = homeg(away, roles) # generate list of away players name
        Teams_code = code(soup_player) # generating list of homw and away team code 
        
        home_team_player.insert(0, Teams_code[0])
        away_team_player.insert(0, Teams_code[1])
        
        full_home_team.append(home_team_player)
        full_away_team.append(away_team_player)
        
        
    except:
        Teams_code_no_player = code(soup_player) # creating team code 
        home_team_player = []
        away_team_player = []
        
        home_team_player.insert(0, Teams_code_no_player[0])
        away_team_player.insert(0, Teams_code_no_player[1])
        
        full_home_team.append(home_team_player)
        full_away_team.append(away_team_player)
        
        
        #print(Teams_code)
    
for i in match_id:
    try:
        url_score = url_root + i
        req_score = requests.get(url_score)
        soup_score = bs(req_score.text, 'html.parser')

        score = scores(soup_score)
        
        score_board.append(score)
    
    except:
        
        score_board.append([0, 0])
  


In [19]:
df2 = pd.DataFrame(full_home_team, columns = ['Home','Hp1','Hp2','Hp3','Hp4','Hp5','Hp6','Hp7','Hp8','Hp9','Hp10','Hp11','Hp12'])
df2
df3 = pd.DataFrame(full_away_team, columns = ['Away','Ap1','Ap2','Ap3','Ap4','Ap5','Ap6','Ap7','Ap8','Ap9','Ap10','Ap11','Ap12','Ap13','Ap14'])
df3
df4 = pd.DataFrame(score_board, columns = ['Home Goal', 'Away Goal'])
del df2['Hp12']
del df3['Ap12']
del df3['Ap13']
del df3['Ap14']



In [20]:
# merging two data frame tgt
data = pd.concat([df2,df3,df4], axis=1)
mid = data['Away']
data.drop(labels=['Away'],axis =1, inplace = True)
data.insert(1, 'Away', mid)


In [21]:
data = data.drop(data.index[249:380])


In [22]:
def result(i):
    if i['Home Goal'] > i['Away Goal']:
        results = 0
    elif i['Home Goal'] == i['Away Goal']:
        results = 1
    else:
        results = 2
    return results
        
data['result'] = data.apply(result, axis=1)

#data.to_csv('league profile.csv')
#data.to_csv(r'C:\Users\nghen\OneDrive\Desktop\Football project 2\league profile.csv')

In [71]:
data = pd.read_csv(r"C:\Users\nghen\OneDrive\Desktop\Football project 2\league profile.csv")
del data['Unnamed: 0']

In [24]:
name_list = []
for i in df['long_name']:
    name_list.append(i)
for i in df['short_name']:
    name_list.append(i)
    
overall_list = []
for i in df['overall']:
    overall_list.append(i)
for i in df['overall']:
    overall_list.append(i)

list_tuple = list(zip(name_list,overall_list))

In [25]:
def score_(special_player_columns):
    name_list_score = []
    for i in data[special_player_columns]:
        for (x,y) in list_tuple:
            if i == x:
                name_list_score.append(y)
                break
        else:
            name_list_score.append(67)
    
    return name_list_score


In [26]:
special_player_columns = list(data.iloc[:,2:24].columns)
player_score_list = []
for i in special_player_columns:
    name_list_score = score_(i)
    player_score_list.append(name_list_score)
    
    




In [27]:
new_player_score = pd.DataFrame(player_score_list).T
home_team_player_score = new_player_score.iloc[:, 0:11].mean(axis=1)
home_team_overall = pd.DataFrame(home_team_player_score, columns = ['Home Team Overall'])
away_team_player_score = new_player_score.iloc[:, 11:].mean(axis=1)
away_team_overall = pd.DataFrame(away_team_player_score, columns = ['Away Team Overall'])
data2 = pd.concat([data,home_team_overall,away_team_overall], axis=1)


In [28]:
data2 = data2.drop(columns = data2.iloc[:,2:24])



In [29]:
data_odd['HomeCode'] = data_odd['HomeCode'].apply(str)
data_odd['AwayCode'] = data_odd['AwayCode'].apply(str)
data_odd['Home vs Away'] = data_odd['HomeCode'] + ' vs ' + data_odd['AwayCode']
data_odd = data_odd.drop(columns = ['HomeCode' , 'AwayCode'])


In [30]:
data2['Home'] = data2['Home'].apply(str)
data2['Away'] = data2['Away'].apply(str)

In [31]:
data2['Home vs Away'] = data2['Home'] + ' vs ' + data2['Away']
data2 = data2.drop(columns = ['Home' , 'Away'])


In [32]:
data_odd_columns = []
for i in data_odd.columns:
    data_odd_columns.append(i)
    
data_odd_columns.remove('Home vs Away')


In [33]:
list_alike =[]
for i in data2['Home vs Away']:
    for x in data_odd['Home vs Away']:
        if i == x:
            #list_alike.append(data_odd[data_odd['Home vs Away'] == x])
            x_list = data_odd[data_odd['Home vs Away'] == x].values.tolist()
            #data_odd[data_odd['Home vs Away'] == x].values.tolist()
            for y in x_list:
                list_alike.append(y)
            

In [34]:
list_alike_df = pd.DataFrame(list_alike)


In [35]:
data_final = pd.concat([data2, list_alike_df], axis = 1)

In [61]:
data_final = data_final.drop([248])

## Splitting Data

In [65]:
X, Y = data_final.iloc[:, 3:], data_final.iloc[:, 2:3]

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, shuffle = True)


## Putting into machine learning 

Model 1: SGD classifier 

In [66]:
from sklearn.linear_model import SGDClassifier 

sgd_clf = SGDClassifier(random_state = 42)
sgd_clf.fit(X_train, Y_train)

C:\Users\nghen\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SGDClassifier(random_state=42)

Testing the accuracy of the Model 1

In [70]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, Y_train, cv=5, scoring = 'accuracy').mean()

C:\Users\nghen\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

0.42897435897435904

Model 2: Random Forest 

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict 

forest_clf = RandomForestClassifier(random_state = 42)
cross_val_score(forest_clf, X_train, Y_train, cv = 5, scoring= 'accuracy').mean()

C:\Users\nghen\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\nghen\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y wa

0.5352564102564102